# 学習モデルを使って実際のレース予想をする

In [76]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

### 予想したいレースのURLを以下に入力　ここのサイトから選ぶ　https://keirin.kdreams.jp/

In [77]:
url = "https://keirin.kdreams.jp/seibuen/racedetail/2620210903030006/"

In [78]:
columns = ['予想', '好気合', '総評', '枠番', '車番', '選手名府県/年齢/期別', '級班', '脚質', 'ギヤ倍数', '競走得点', '1着', '2着', '3着', '着外']

### データを取ってくる

In [79]:
race_data = pd.read_html(url)[4][:-1]
race_data.columns = columns
race_data = race_data.astype(str)
time.sleep(1)

### データを加工する

In [80]:
race_data = pd.concat([race_data, race_data["選手名府県/年齢/期別"].str.split("/", expand=True)], axis=1).drop("選手名府県/年齢/期別", axis=1)
race_data = race_data.drop(0, axis=1)

race_data["予想"] = pd.Categorical(race_data["予想"], categories=["nan", "×", "▲", "△", "○", "◎", "注"])
race_data["好気合"] = pd.Categorical(race_data["好気合"], categories=["★"])
race_data["脚質"] = pd.Categorical(race_data["脚質"], categories=["両", "追", "逃"])
race_data["級班"] = pd.Categorical(race_data["級班"], categories=["A1", "A2", "A3", "L1", "S1", "S2", "SS"])

prediction = pd.get_dummies(race_data["予想"])
isFine = pd.get_dummies(race_data["好気合"])
leg_type = pd.get_dummies(race_data["脚質"])
group = pd.get_dummies(race_data["級班"])

race_data = pd.concat([race_data, prediction, isFine, leg_type, group], axis=1).drop(["予想", "好気合", "脚質", "級班"], axis=1)
print(race_data.columns)
race_data = race_data.replace(["失", "落", "故", "欠"], 9)
race_data["ギヤ倍数"] = race_data["ギヤ倍数"].map(lambda x: x[:4] if len(x)>4 else x)
race_data[2] = race_data[2].map(lambda x: x.replace(" （欠車）", "") if "欠車"in x else x)
race_data = race_data.astype("float64")
race_data = race_data.loc[:, ['車番', '総評', '枠番', 'ギヤ倍数', '競走得点', '1着', '2着', '3着', '着外', 1, 2, 'nan', '×', '▲', '△', '○', '◎', '注', '★', '両', '追', '逃', 'A1', 'A2', 'A3', 'L1', 'S1', 'S2', 'SS']]
race_data

Index([  '総評',   '枠番',   '車番', 'ギヤ倍数', '競走得点',   '1着',   '2着',   '3着',   '着外',
            1,      2,  'nan',    '×',    '▲',    '△',    '○',    '◎',    '注',
          '★',    '両',    '追',    '逃',   'A1',   'A2',   'A3',   'L1',   'S1',
         'S2',   'SS'],
      dtype='object')


,車番,総評,枠番,ギヤ倍数,競走得点,1着,2着,3着,着外,1,...,両,追,逃,A1,A2,A3,L1,S1,S2,SS
0,1.0,4.0,1.0,3.92,84.11,2.0,5.0,10.0,33.0,44.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2.0,5.0,2.0,3.92,77.80,0.0,0.0,4.0,33.0,52.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,3.0,3.0,3.92,87.00,7.0,10.0,17.0,35.0,44.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,4.0,4.0,3.92,81.16,2.0,2.0,2.0,20.0,42.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5.0,4.0,5.0,3.92,81.50,0.0,3.0,4.0,23.0,47.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,6.0,7.0,6.0,3.92,0.00,5.0,6.0,7.0,16.0,37.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7.0,4.0,6.0,3.93,80.81,2.0,4.0,9.0,42.0,44.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### 学習済みモデルに入れて予想する

In [81]:
import tensorflow as tf
from tensorflow import keras

In [82]:
model = tf.keras.models.load_model("pred_top3")

In [83]:
race_data.shape

(7, 29)

In [84]:
results = model.predict(race_data)

In [85]:
results

array([[0.6933929 ],
       [0.3798601 ],
       [0.75922304],
       [0.5098343 ],
       [0.3202281 ],
       [0.10284325],
       [0.22843379]], dtype=float32)